### 5.3 多输入通道和多输出通道

> **真实彩色图像中,除高`h`和宽`w`2个维度外,还有RGB(红,绿,蓝)3个颜色通道,可用 $3 \times h \times w$表示,此时将大小为3的这一维度成为通道(channel)**

### 5.3.1 多输入通道

> 1. **假设输入数据通道数为$c_i$,那卷积核同样为$c_i$,且大小为$k_h \times k_w$,此时卷积核形状为$c_i \times k_h \times k_w$**
> 2. **在输入数据和卷积核对应的每个通道上,进行互相关运算,最后将$c_i$个互相关运算的二维输出按通道相加**

In [1]:
import torch
from torch import nn
import sys
sys.path.append("..")
import d2lzh_pytorch.utils as d2l

In [2]:
def corr2d_multi_in(X, K):
    # 分别沿着X和K的第0维（通道维）分别计算在相加
    res = d2l.corr2d(X[0, :, :], K[0, :, :])
    for i in range(1, X.shape[0]):
        res += d2l.corr2d(X[i, :, :], K[i, :, :])
    return res

In [3]:
X = torch.tensor([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
              [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = torch.tensor([[[0, 1], [2, 3]], [[1, 2], [3, 4]]])

print(X.shape)
print(K.shape)

corr2d_multi_in(X, K)

torch.Size([2, 3, 3])
torch.Size([2, 2, 2])


tensor([[ 56.,  72.],
        [104., 120.]])

### 5.3.2 多输出通道

> 1. **设卷积核的输入和输出通道分别为$c_i$和$c_o$,大小为$k_h \times k_w$**
> 2. **为每个输出通道分别创建形状为$c_i \times k_h \times k_w$的卷积核,将他们在输出通道上连结,此时卷积核的形状为$c_o \times c_i \times k_h \times k_w$, 即（输出通道，输入通道，高， 宽）**
> 3. **每个输出通道上的结果,由上述卷积核在该输出通道上的核数组与整个输入数组计算得来**

In [5]:
def corr2d_multi_in_out(X, K):
    # 对K第0维遍历,每次与X做互相关运算,最后结果使用stack结合在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K])

In [6]:
# 使用K, K+1, K+2 连结在一起构造一个输出通道为3的卷积核
K = torch.stack([K, K+1, K+2])
K.shape

torch.Size([3, 2, 2, 2])

In [7]:
# 第一个通道的结果与上面单输出结果一致
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

### 5.3.3  1 X 1 卷积层

> 1. **主要用于通道维计算,此时输入和输出具有相同的宽和高**
> 2. **输出元素等于输入元素在相同宽、高位置上元素在不同位置的按权累加**
> 3. **假设将通道维当做特征维,将宽高维度上的元素当做样本,那么1X1卷积的作用等价于全连接**

In [28]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.view(c_i, h * w)
    K = K.view(c_o, c_i)
    Y = torch.mm(K, X)
    return Y.view(c_o, h, w)

In [29]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.view(c_i, h * w)
    K = K.view(c_o, c_i)
    Y = torch.mm(K, X)  # 全连接层的矩阵乘法
    return Y.view(c_o, h, w)

In [36]:
X = torch.rand(3, 3, 3)
# K此时是2输出通道,3输入通道,1x1
K = torch.rand(2, 3, 1, 1)

# 变换为 (2, 3) . (3, 9) => (2, 9)
# 最后在变为(2, 3, 3)

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
# 这个为了说明两者效果一致
(Y1 - Y2).norm().item() < 1e-6

True

In [34]:
K.view(2, 3)

tensor([[0.1673, 0.7500, 0.8406],
        [0.4102, 0.1670, 0.8739]])

In [35]:
X.view(3, 9)

tensor([[0.7067, 0.5082, 0.2194, 0.4368, 0.7227, 0.4741, 0.3642, 0.7392, 0.7249],
        [0.9680, 0.8728, 0.1763, 0.8929, 0.5334, 0.5359, 0.7170, 0.6469, 0.0583],
        [0.0072, 0.5873, 0.9747, 0.9591, 0.5170, 0.1785, 0.1421, 0.3002, 0.1103]])